# Exploration of ducklib library

First thing first, let's see that the library is installed and we are able to run it.

In [ ]:
import ducklib

In [ ]:
ducklib.say_hi()

We are going to look a bit at out library with the help of [inspect](https://docs.python.org/3/library/inspect.html).

In [ ]:
import inspect

In [ ]:
inspect.getsource(ducklib.main.say_hi)

Interesting, we see that username and password are read from the environment and then there is `auth.is_user_valid` call. Let's look at it. But first we need to figure out where this function is.

In [ ]:
dir(ducklib)

We see that `ducklib` provides three functions or modules:

1. `auth`
2. `auth_functions`, that one is just for illustrative purposes as I will show two ways of tampering.
3. `main`


In [ ]:
dir(ducklib.auth)

In [ ]:
inspect.getsource(ducklib.auth.is_user_valid)

OK. `inspect` reports that this is a builtin function or method and that it can't get it's source code. This is expected.

We may get some more details about the function of interest (*):

In [ ]:
inspect.getfullargspec(ducklib.auth.is_user_valid)

This tells us a lot!

* The list of arguments!
* Their default values!
* Their types! (unicode)
* Return value!

Knowing all this, tampering the library is really easy!

## Atack methods

There are two methods to alter `ducklib` behaviour:

1. Substitution of library compiled files.
2. Purely python-based.

### Attack method #1

Our attack will be aimed at `auth.is_user_valid` function. We will create our own malicious `auth` module and place it instead of original one in the virtual environment.

Let's head towards the folder `ducklib-jammed` of the project.

**Remember to restart kernel if you modify the library in the virtual enviroment!** This is needed to reload it from disc into memory.

### Automation of file replacement, use with caution

If you have your virtual environments in `venv` folder for each of the `ducklib` repos, then you can use the script below to modify `auth` module.

In [ ]:
!cd ../ducklib-jammed && %CD%\venv\Scripts\activate.bat && pip install cython~=0.29.16 && python setup.py bdist_wheel 
!copy %CD%\..\ducklib-jammed\build\lib.win-amd64-3.7\ducklib\auth.cp37-win_amd64.pyd %CD%\venv\Lib\site-packages\ducklib\auth.cp37-win_amd64.pyd

#### After library tampering

Auth module is replaced, kernel has been rerstarted. We are ready to proceed.

In [ ]:
import ducklib

In [ ]:
ducklib.main.say_hi()

### Attack method #2

Please restart the kernel!

This is python-only method. In Python, we may reload functions at run-time. It is really easy to write wrappers/adapters, e.t.c.

The success of this method may depend on how original `ducklib` library imports modules. (note to self: really?)

This method will alter `auth_functions`. This module contains an additional function `configure()`. We do not want to modify that function, but rather to call the original one.

In [ ]:
import ducklib.auth_functions as auth_real

In [ ]:
def my_validation(username='', pwd='') -> bool:
    return True

configure_original = auth_real.configure
def my_configure():
    print('Wrapper for real configure()')
    return configure_original()

In [ ]:
# change binding
auth_real.is_user_valid = my_validation
auth_real.configure = my_configure

In [ ]:
# it's time to import, however auth_functions is already in the memory and won't be reloaded
import ducklib

In [ ]:
ducklib.start_chatting()

## Closing remarks

In [ ]:
import ducklib.auth
type(ducklib.auth.is_user_valid)

Observe that the type of function is `cython_function_or_method`. This was essential for getting `inspect` working. However, Cython can compile functions as `built-in`. In such cases, `inspect` won't be able to extract any information about the function.

Let's compare it with library from repo `ducklib-builtin`. Install ducklib from `ducklib-builtin/dist`, proceed.

In [ ]:
import ducklib_builtin.auth
type(ducklib_builtin.auth.is_user_valid)

In [ ]:
import inspect
inspect.getfullargspec(ducklib_builtin.auth.is_user_valid)

Can we still apply Python-based attack? Restart kernel.

In [ ]:
import ducklib_builtin.auth_functions as auth_real
def my_validation(username='', pwd='') -> bool:
    return True

configure_original = auth_real.configure
def my_configure():
    print('Wrapper for real configure()')
    return configure_original()
# change binding
auth_real.is_user_valid = my_validation
auth_real.configure = my_configure

In [ ]:
import ducklib_builtin
ducklib_builtin.start_chatting()

Sweet! How about the code below? (restart kernel)

In [ ]:
import ducklib_builtin.auth_functions as auth_real
def my_validation(username='') -> bool:
    return True

configure_original = auth_real.configure
def my_configure():
    print('Wrapper for real configure()')
    return configure_original()
# change binding
auth_real.is_user_valid = my_validation
auth_real.configure = my_configure

import ducklib_builtin
ducklib_builtin.start_chatting()

Ouch, we got information about the arguments, so we can now write correct wrapper.